# Step-by-step tutorial

**Scope:**  

This tutorial shows how to use cryoswath to process CryoSat-2 SARIn L1b
data to gridded trends of elevation change using the default setting.
All processing steps are triggered manually to show what usually happens
in the background.

First things first: if you haven't done so, install the packages listed
in [`../requirements.txt`](../requirements.txt). The required reference
DEM and shape files for this tutorial are provided, however, note that
this will likely not be sufficient for your own research.

This tutorial will use Barnes Ice Cap in the Southern Canadian Arctic as study region.

In [1]:
import os

In [2]:
tutorial_aux_data_path = os.path.join("..", "data", "tutorials")

In [3]:
dem_file_path = os.path.join(tutorial_aux_data_path, "arcticdem_mosaic_100m_v4.1_dem__excerpt_barnes-ice-cap.tif")

In [4]:
import geopandas as gpd

In [ ]:
# load barnes ice cap outline
barnes_shp = gpd.read_feather(os.path.join(tutorial_aux_data_path, "barnes_ice_cap.feather")).unary_union

Next, CryoSat-2 SARIn ground tracks, that pass in some defined period
close to Barnes Ice Cap, will be loaded. You can best reduce the
downloaded and processed amount of data, by editing the start and end
dates below.

In [6]:
# make the package available/from search path
import sys
sys.path.insert(0, "..")

from cryoswath import misc  # this module contains helper functions

from dateutil.relativedelta import relativedelta


KeyboardInterrupt: 

In [ ]:
example_tracks = misc.load_cs_ground_tracks(
    barnes_shp, "2020-09", "2023-09", # request region and period
    buffer_period_by=relativedelta(months=1), # include previous and following months
    buffer_region_by=5_000) # include basin margins
print("number of tracks:", example_tracks.shape[0], "\n last five:", example_tracks.tail(5))

This `pandas.Series` contains the CryoSat-2 tracks. However, what we are
more interested in, here, is the Index of the Series which can be used
to download the associated L1b files.

Note: You could as well immediately pass the region definitions to the
download function - in fact, you could jump all the steps below. This
detour is only for the purpose of this tutorial. 

In [8]:
from cryoswath import l1b

In [9]:
# # check whether L1b files are present and download if they aren't
# l1b.download_wrapper(track_idx=example_tracks.index)

## Level L1b to L2 processing

L1b describes a data stage where the data have undergone some
preprocessing but relate only indirectly to the observable world.

L2 refers to data that describe physical "real-world" properties. This
will mainly be the surface elevation at various locations, here.

Let's look at the first of the tracks.

You can load the L1b data by either using the index of the concerned
track.

In [ ]:
example_l1b_data = l1b.L1bData.from_id(example_tracks.index[0], drop_outside=barnes_shp, smooth_phase_difference=True)

Internally, first groups of continuous samples have to be recognized.

In [11]:
example_l1b_data = example_l1b_data.tag_groups()

With the groups identified, you can "unwrap" the phase difference.
(Unwrapping is like realizing that a series of time stamps: 11 pm, 12
pm, 1 am extends over two days.)

In [12]:
example_l1b_data = example_l1b_data.unwrap_phase_diff()

Afterward, you can have the reference elevations calculated for each
apparent echo origin. For this, obviously, one need a reference
elevation dataset. For this tutorial, it is included.

In [13]:
dem_file_path = os.path.join(tutorial_aux_data_path, "arcticdem_mosaic_100m_v4.1_dem__excerpt_barnes-ice-cap.tif")

In [14]:
example_l1b_data = example_l1b_data.append_ambiguous_reference_elevation(dem_file_path)

cryoswath frequently uses the elevation difference of the echo origin
with regard to the reference dataset. Below, this value is stored.

In [15]:
example_l1b_data = example_l1b_data.append_elev_diff_to_ref()

Finally, for each group, the best fitting of the ambiguous echo origins
is found based on a statistical measure. Below, the default is made
explicit. As long as you prefer the default statistic, you don't need to
pass the argument. However, it shows how you could define any custom
measure you deem appropriate.

In [16]:
import numpy as np
from scipy.stats import median_abs_deviation
def the_default(elev_diff):
    return np.argmin(np.abs(np.median(elev_diff, axis=0))**2+median_abs_deviation(elev_diff, axis=0)**2)

In [17]:
example_l1b_data = example_l1b_data.append_best_fit_phase_index(best_column=the_default)

Finally, we visualize the result:  
*(The points in different shades of gray indicate other mathematically
valid echo origins.)*

In [18]:
from cryoswath.test_plots.waveform import dem_transect

In [ ]:
dem_transect(example_l1b_data.isel(time_20_ku=[len(example_l1b_data.time_20_ku)//2]), dem_file_name_or_path=dem_file_path, selected_phase_only=False)#, ax=ax

**Final remarks L1b to L2**

This were the most important steps in the L1b to L2 processing. Usually,
you will rather use shorthands to have the steps done above
automatically. However, if you have reasons to do so, this is how you
can trigger all the major steps manually.

## L2 to L3 processing

L3 data features slots for a set of locations and time steps. For our
case, L3 data refers to gridded monthly elevation data.

In [20]:
from cryoswath import l2

First, all the tracks previously are processed - now, using a shorthand.

The arguments, usually, are not necessary. Here, they set the caching
location to the tutorial directory, confine the processed data to the
study region, pass the elevation model, set a threshold for the maximum
disagreement wrt. the reference DEM, prohibit parallelism (often a good
idea on notebooks with limited working memory), define a coordinate
reference system (CRS) for the output locations, and make explicit that
all data is processed, even if it is present. The last option, you may
want to disable if you know that the tracks have not been processed with
different settings (e.g., you run the cell for the second time).

*note:* should you do the l1b to l2 processing below in multiple
        sessions (takes about 30 min in total), the returned
        `swath_data, poca_data` will only contain the part processed in
        the last run. Should you want to load all data, use the
        commented cell below (only works once you've run the first).

In [21]:
swath_data, poca_data = l2.from_id(example_tracks.index, cache_fullname=os.path.join(tmp_path, "tmp_l2_cache"), drop_outside=barnes_shp, dem_file_name_or_path=dem_file_path, max_elev_diff=150, cores=1, crs=3413, reprocess=True)

In [22]:
# # takes a few minutes (2-3 min on my notebook)
# empty_shape = gpd.GeoSeries().union_all("coverage")
# swath_data, poca_data = l2.from_id(example_tracks.index, reprocess=False, 
#                                    save_or_return="return", # only return data (do not save to disk)
#                                    drop_outside=empty_shape, # indirectly: do not process any files
#                                    )

Then, we take a look at the data by producing histograms.

In [ ]:
import matplotlib.pyplot as plt

In [23]:
swath_data.h_diff.plot.hist(label=f"swath, Σ={len(swath_data):_}", bins=100, density=True)
poca_data.h_diff.plot.hist(label=f"POCA, Σ={len(poca_data):_}", bins=100, density=True, ax=plt.gca(), alpha=0.8)
plt.legend()

Finally, we aggregate the point data into a regular grid. You will
recognize most of the arguments from the previous steps. However, I
would like to highlight the `agg_func_and_meta` argument. This allows to
pass a custom aggregation function. Be careful modifying the aggregates
names, because as of now (Feb 2025) those names are hardcoded in other
functions. So feel free to play around, but keep the names if possible.

In [24]:
from cryoswath import l3

In [25]:
import pandas as pd

In [26]:
def med_iqr_cnt(data):
    quartiles = np.quantile(data, [.25, .5, .75])
    return pd.DataFrame([[
        quartiles[1],  # the median
        quartiles[2]-quartiles[0],  # the inter-quartile range
        len(data)  # the data count
    ]], columns=["_median", "_iqr", "_count"])

In [ ]:
# takes a few minutes (3-4 min at 4 GHz)
gridded_elev_diffs = l3.build_dataset(
    barnes_shp, start_datetime="2020-09", end_datetime="2023-09",
    cache_filename="tmp_l2_cache",
    drop_outside=barnes_shp,
    buffer_region_by=5_000,
    crs=3413,
    reprocess=False,
    timestep_months = 1,  # temporal resolution
    window_ntimesteps = 3,  # aggregates multiple time steps (moving window)
    spatial_res_meter = 500,  # size of the grid cells
    # below, you need to disclose the returned format
    agg_func_and_meta = (med_iqr_cnt, {"_median": "f4", "_iqr": "f4", "_count": "i4"}),
).load()

Should you see warnings about dumped data in the output above, don't
worry. This usually concerns only data outside of the glaciers and it
can be recovered using a helper function that collects the dumps. I'll
fix it (#37) after finishing the tutorial.

# L3 to L4 processing

L4 data may contain inferred information. In our case, that concerns
void filling and calculating change rates.

First, we do a rudimentary data clean-up.

In [28]:
gridded_elev_diffs["_median"] = gridded_elev_diffs._median.where(gridded_elev_diffs._count>3).where(gridded_elev_diffs._iqr<30)

From there, the elevation change rate can be fitted.

In [29]:
from cryoswath import l4

In [ ]:
gridded_fit_results = l4.fit_trend__seasons_removed(gridded_elev_diffs)

Before later we can fill the voids, we should exclude outliers to
prevent them from distorting the data distribution. Here, I do so by
requiring the trend variance to be less than 2 m²/yr² and the amplitude
variances of the annual and semi-annual cycle to be smaller than 100 m².
This corresponds roughly to limiting the trend and amplitude 2-sigma
uncertainties to 3 m/yr and 20 m, respectively.

For clarity, below the new "trend" and "trend_std" variables are
defined.

In [31]:
import numpy as np

In [32]:
mask = np.logical_and(
    gridded_fit_results.curvefit_covariance.sel(cov_i="trend", cov_j="trend") < 2,
    np.logical_and(gridded_fit_results.curvefit_covariance.sel(cov_i="amp_yearly", cov_j="amp_yearly") < 100,
                    gridded_fit_results.curvefit_covariance.sel(cov_i="amp_semiyr", cov_j="amp_semiyr") < 100)
)

In [33]:
gridded_trends = gridded_fit_results.where(mask).sel(param="trend", cov_i="trend", cov_j="trend")
gridded_trends["trend"] = gridded_trends.curvefit_coefficients
gridded_trends["trend_std"] = gridded_trends.curvefit_covariance**.5

We, then, ensure that there is no missing grid cells. This used to occur
before I moved to Zarr for regions with larger gaps between glaciers. If
grid cells are missing, this messes up the apparent resolution as seen
by `rioxarray` that is used in the background.

Further, we cut the data that is outside of the glacier.

In [34]:
barnes_GeoDataFrame_SRID3413 = gpd.read_feather(os.path.join(tutorial_aux_data_path, "barnes_ice_cap.feather")).to_crs(gridded_trends.rio.crs)

In [35]:
gridded_trends = l3.fill_missing_coords(gridded_trends, *barnes_GeoDataFrame_SRID3413.total_bounds)
gridded_trends = gridded_trends.rio.clip(barnes_GeoDataFrame_SRID3413.make_valid())

Now, we append the elevation of a reference DEM and assign the grid
cells to basins. Actually, here we do *not* consider the basins, but
treat the entire ice cap collectively.

In [36]:
gridded_trends = l3.append_elevation_reference(gridded_trends.rio.clip(barnes_GeoDataFrame_SRID3413.geometry), dem_file_name_or_path=dem_file_path)

In [37]:
gridded_trends = l3.fill_voids(gridded_trends[["trend", "trend_std", "ref_elev"]],
                               main_var="trend",
                               error="trend_std",
                               elev="ref_elev",
                               basin_shapes=barnes_GeoDataFrame_SRID3413,
                               per=tuple(), outlier_replace=True, outlier_limit=2)

In [38]:
import matplotlib.pyplot as plt

In [ ]:
gridded_trends.trend.T.plot(robust=True, cmap="RdYlBu")
plt.gca().set_aspect("equal")

*Regarding the above figure*  
There are artifacts, showing as patches of strong elevation gains and
losses (blue and red) at higher elevations along the ice divide at the
center of the ice cap. You could use the `dem_transect` plotting
function from above to investigate their origin. Some issues are known
to especially affect data covering only less than 4 years and to occur
around summits (web-search: systematic error cryosat swath).

### Time series

**coming soon**

Calculating time series works in analogy to the above. However, this
tutorial works on a RGI "complex", here, the Barnes Ice Cap, while I
usually work on "basins", i.e., constituting parts of complexes. I need
to adapt the code a bit to work on complexes as well.